# Use GLM-4 API to fine-tune model with LORA method

**This tutorial is available in English and is attached below the Chinese explanation**

此代码将展示如何调用 GLM-4 API 来使用LORA的方法对一个 glm 模型进行微调。你将会在本代码中学到：
1. 微调数据集的格式
2. 如何上传微调的数据集
3. 如何管理和查看自己的微调数据集
4. 如何设置超参数并开始微调模型
5. 如何调用微调后的模型

This code will show how to call the GLM-4 API to fine-tune a glm model using LORA methods. What you will learn in this code:
1. Fine-tune the format of the data set
2. How to upload fine-tuned dataset
3. How to manage and view your own fine-tuning data set
4. How to set hyperparameters and start fine-tuning the model
5. How to call the fine-tuned model

首先，我们设置好 API KEY，并将这个API KEY设置为环境变量，这样我们就可以在代码中直接调用了。

First, we set up the API KEY and set this API KEY as an environment variable so that we can call it directly in the code.

In [1]:
import os
from zhipuai import ZhipuAI

os.environ["ZHIPUAI_API_KEY"] = "your api key"
client = ZhipuAI()

## Prepare the dataset

需要准备训练的数据集，并将其拆分为训练集和验证集。我是用 [AdvertiseGen 数据集](https://drive.google.com/file/d/13_vf0xRTQsyneRKdD1bZIr93vBGOczrk/view) 并将其对其和自动拆分成了训练集和验证集。
我们先下载数据集并将其放到 `data` 文件夹下并解压缩，接着运行如下脚本来转换数据集格式。
由于 Zhipu AI的数据集格式必须按照[官方文档](https://open.bigmodel.cn/dev/howuse/finetuning)的格式进行编排,因此，整理完后的数据集格式如下：

The training data set needs to be prepared and split into a training set and a validation set. I used the [AdvertiseGen data set](https://drive.google.com/file/d/13_vf0xRTQsyneRKdD1bZIr93vBGOczrk/view) and automatically split it into a training set and a validation set.
We first download the data set and place it in the `data` folder and unzip it, then run the following script to convert the data set format.
Since the data set format of Zhipu AI must be arranged according to the [official document](https://open.bigmodel.cn/dev/howuse/finetuning), the format of the completed data set is as follows:

```
{"messages": [{"role": "user", "content": "类型#裤*版型#宽松*风格#性感*图案#线条*裤型#阔腿裤"}, {"role": "assistant", "content": "宽松的阔腿裤这两年真的吸粉不少，明星时尚达人的心头爱。毕竟好穿时尚，谁都能穿出腿长2米的效果宽松的裤腿，当然是遮肉小能手啊。上身随性自然不拘束，面料亲肤舒适贴身体验感棒棒哒。系带部分增加设计看点，还让单品的设计感更强。腿部线条若隐若现的，性感撩人。颜色敲温柔的，与裤子本身所呈现的风格有点反差萌。"}]}
{"messages": [{"role": "user", "content": "类型#裙*风格#简约*图案#条纹*图案#线条*图案#撞色*裙型#鱼尾裙*裙袖长#无袖"}, {"role": "assistant", "content": "圆形领口修饰脖颈线条，适合各种脸型，耐看有气质。无袖设计，尤显清凉，简约横条纹装饰，使得整身人鱼造型更为生动立体。加之撞色的鱼尾下摆，深邃富有诗意。收腰包臀,修饰女性身体曲线，结合别出心裁的鱼尾裙摆设计，勾勒出自然流畅的身体轮廓，展现了婀娜多姿的迷人姿态。"}]}

```

In [2]:
from typing import Union
from pathlib import Path
import json


def advertise_gen_convert(data_dir: Union[str, Path], save_dir: Union[str, Path]):
    def _convert(in_file: Path, out_file: Path):
        with open(in_file, encoding='utf-8') as fin, open(out_file, 'w', encoding='utf-8') as fout:
            for line in fin:
                item = json.loads(line)
                sample = {'messages': [{'role': 'user', 'content': item['content']},
                                       {'role': 'assistant', 'content': item['summary']}]}
                fout.write(json.dumps(sample, ensure_ascii=False) + '\n')

    data_dir = Path(data_dir).resolve()
    save_dir = Path(save_dir).resolve()

    for file_name in ['train.json', 'dev.json']:
        in_file = data_dir / file_name
        out_file = save_dir / f'{in_file.stem}.jsonl'

        if in_file.is_file():
            out_file.parent.mkdir(parents=True, exist_ok=True)
            _convert(in_file, out_file)


advertise_gen_convert(data_dir="data/AdvertiseGen", save_dir="data/AdvertiseGen_Formatted")

将数据集转换结束后，通过SDK提供的接口上传数据集到Zhipu AI的服务器上，并查看上传的数据集的文件列表。

After the data set is converted, upload the data set to Zhipu AI's server through the interface provided by the SDK, and view the file list of the uploaded data set.

In [3]:
client.files.create(
    file=open("data/AdvertiseGen_Formatted/train.jsonl", "rb"),
    purpose="fine-tune"
)

client.files.create(
    file=open("data/AdvertiseGen_Formatted/dev.jsonl", "rb"),
    purpose="fine-tune"
)
client.files.list()

ListOfFileObject(object='list', data=[FileObject(id='file-20240126055309882-wnpcq', bytes=60066225, created_at=1706248392000, filename='train.jsonl', object='fd', purpose='fine-tune', status=None, status_details=None), FileObject(id='file-20240126055312192-6zlhw', bytes=557244, created_at=1706248392000, filename='dev.jsonl', object='fd', purpose='fine-tune', status=None, status_details=None), FileObject(id='file-20240126032709833-5zrkr', bytes=557244, created_at=1706239630000, filename='dev.jsonl', object='fd', purpose='fine-tune', status=None, status_details=None), FileObject(id='file-20240126032453929-hm9mh', bytes=60066225, created_at=1706239496000, filename='train.jsonl', object='fd', purpose='fine-tune', status=None, status_details=None)], has_more=None)

## Start training using the uploaded file

现在，在列表中已经找到了训练集和验证集的两个文件，我们设置好相关的参数，并上传到 Zhipu AI 的服务器上开始训练。按照[官方文档](https://open.bigmodel.cn/dev/api#finetuning)填写相关参数。我们就可以开训练了。我选择使用最小的模型 `chatglm3-6b` 进行训练。

<span style='color: red; font-weight: bold;'>⚠️ 当前 ZhipuAI API 还不支持 更大参数的模型训练，训练方法也仅支持 LORA ⚠️</span>

当前版本SDK中，默认使用的是 LORA 的方式进行训练。由于超参数在文档中仅开放了三个参数，分别是
- `learning_rate_multiplier` 学习率的倍数
- `batch_size` 批次大小
- `n_epochs` 训练轮数

接着，我们就能开始训练了，并且，我们打印了训练的`job_id`，以便后续查看训练进度。


Now that we have found the two files of the training set and validation set in the list, we set the relevant parameters and uploaded them to the Zhipu AI server to start training. Fill in the relevant parameters according to [official documentation](https://open.bigmodel.cn/dev/api#finetuning). We can start training. I chose to use the smallest model `chatglm3-6b` for training.

<span style='color: red; font-weight: bold;'>⚠️ Currently ZhipuAI API does not support model training with larger parameters, and the training method only supports LORA ⚠️</span>

In the current version of the SDK, the LORA method is used for training by default. Since the hyperparameters only open three parameters in the document, they are
- `learning_rate_multiplier` multiple of the learning rate
- `batch_size` batch size
- `n_epochs` Number of training epochs
These values have default values in the documentation, but I chose to set them myself.


In [4]:
# set more hyperparameters
# job = client.fine_tuning.jobs.create(
#     model="chatglm3-6b",
#     training_file="file-20240126055312192-6zlhw",
#     validation_file="file-20240126055312192-6zlhw",
#     hyperparameters={
#         "learning_rate_multiplier": 0.5,
#         "batch_size": 32,
#         "n_epochs": 2,
#     },
#     suffix="advertise_gen",
# )

# using default hyperparameters
job = client.fine_tuning.jobs.create(
    model="chatglm3-6b",
    training_file="file-20240126055312192-6zlhw",
    validation_file="file-20240126055312192-6zlhw",
)
job_id = job.id

## 查看训练进度
我们可以查看这个训练任务的进度，以及训练的状态。 由于训练任务是异步的，因此，我们需要等待一段时间后才能看到训练的状态。其中
- `status` 训练状态，现在是 `running`，表示正在训练

由于本数据集的训练机数量较大（超过11万条数据），训练时间较长（在书写本cookbook时微调时间超过了1个小时), 因此，我在实际微调的过程使用，使用的是 `dev.jsonl` 文件进行微调和验证
您可以自己删除部分数据集，这样能有效降低训练时间和训练的token消耗成本。

We can check the progress of this training task and the status of training. Since the training task is asynchronous, we need to wait for a period of time before we can see the status of the training. in
- `status` training status, now `running`, indicating training is in progress

Since the number of training machines in this data set is large (more than 110,000 pieces of data) and the training time is long (the fine-tuning time took more than 1 hour when writing this cookbook), therefore, in the actual fine-tuning process, I used `dev.jsonl` file for fine-tuning and verification
You can delete part of the data set yourself, which can effectively reduce training time and training token consumption costs.

In [5]:
fine_tuning_job = client.fine_tuning.jobs.retrieve(fine_tuning_job_id=job_id)

id='ftjob-20240126135812634-4m65v' request_id=None created_at=None error=None fine_tuned_model='ft:chatglm3-6b:advertise_gen:sg59pfxk' finished_at=None hyperparameters=None model='chatglm3-6b' object='fine_tuning.job' result_files=[] status='running' trained_tokens=None training_file='file-20240126055312192-6zlhw' validation_file='file-20240126055312192-6zlhw'


通过等待一段时间我们再次查询，我们发现，模型已经训练完成，接下来，我们就是调用模型进行测试了。
通过 Python SDK来查询我所有的微调任务进度，其中，这些字段的含义如下：
- `id` 微调任务的id
- `fine_tuned_model` 微调模型的名称
- `request_id` 用户请求的id，默认是空
- `created_at`,`finished_at` 任务的开始时间和结束时间,在这里都是空
- `status` 任务的状态，现在是 `running`，表示正在训练

By waiting for a while and querying again, we find that the model has been trained. Next, we call the model for testing.
Use the Python SDK to check the progress of all my fine-tuning tasks.

In [18]:
client.fine_tuning.jobs.list()

ListOfFineTuningJob(object='list', data=[FineTuningJob(id='ftjob-20240126135812634-4m65v', request_id=None, created_at=None, error=None, fine_tuned_model='ft:chatglm3-6b:advertise_gen:sg59pfxk', finished_at=None, hyperparameters=None, model='chatglm3-6b', object='fine_tuning.job', result_files=['file-20240126060112746-zqkss'], status='succeeded', trained_tokens=None, training_file='file-20240126055312192-6zlhw', validation_file='file-20240126055312192-6zlhw'), FineTuningJob(id='ftjob-20240126113041678-cs6s9', request_id=None, created_at=None, error=None, fine_tuned_model='ft:chatglm3-6b:advertise_gen:dkgf9mc8', finished_at=None, hyperparameters=None, model='chatglm3-6b', object='fine_tuning.job', result_files=[], status='failed', trained_tokens=None, training_file='file-20240126032453929-hm9mh', validation_file='file-20240126032709833-5zrkr')], has_more=None)

## 调用微调后的模型

在完成微调任务之后，我们会得到一个新的模型，我们可以通过SDK来完成对这个模型的调用。我们首先查看一下模型列表，然后选择最新的模型进行调用。使用微调后的模型的请求方式与常规的 GLM 模型没有区别，仅需将 `model` 更换为刚才微调的模型名称即可。

After completing the fine-tuning task, we will get a new model, and we can complete the call to this model through the SDK. We first check the model list, and then select the latest model to call. The request method for using the fine-tuned model is no different from the regular GLM model. You only need to replace `model` with the name of the model you just fine-tuned.


In [19]:
response = client.chat.completions.create(
    model="ft:chatglm3-6b:advertise_gen:sg59pfxk",
    messages=[
        {"role": "user", "content": "类型#裤*风格#英伦*风格#简约"},
    ],
)
response.choices[0].message.content

'这件裤子采用了简约的英伦风格设计，展现出了古典与现代的完美结合。它采用了合身的剪裁，让您的身形更加挺拔优雅。裤子的面料使用了高品质的棉质面料，让您在穿着时更加舒适自在。此外，这条裤子还采用了细节处理，如精细的缝线和纽扣，展现了品质卓越的特点。整体来看，这条裤子既简约又具有时尚感，是您英伦风格搭配的理想选择。'